In [ ]:
!pip install rdflib

import pandas as pd
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD

# Caminhos para os arquivos CSV
csv_paths = {
    '2015': '/content/drive/MyDrive/Mestrado/TrabalhoESeBD/microdados_filtrados/microdados2015_filtrados.csv',
    '2018': '/content/drive/MyDrive/Mestrado/TrabalhoESeBD/microdados_filtrados/microdados2018_filtrados.csv',
    '2022': '/content/drive/MyDrive/Mestrado/TrabalhoESeBD/microdados_filtrados/microdados2022_filtrados.csv'
}

# Carregar ontologia OWL existente
ontology_path = '/content/drive/MyDrive/Mestrado/TrabalhoESeBD/ontologia/enade.owl'
g = Graph()
try:
    g.parse(ontology_path, format='xml')
except Exception as e:
    print(f"Erro ao carregar ontologia: {e}")

# Definir o namespace da ontologia
ENDE_NAMESPACE = Namespace('http://example.org/ontologies/enade#')
g.bind('enade', ENDE_NAMESPACE)

# Função para adicionar instâncias à ontologia
def add_instance(graph, class_name, individual_id, properties):
    individual_uri = ENDE_NAMESPACE[individual_id]
    graph.add((individual_uri, RDF.type, ENDE_NAMESPACE[class_name]))
    for prop, value in properties.items():
        if isinstance(value, str):
            graph.add((individual_uri, ENDE_NAMESPACE[prop], Literal(value, datatype=XSD.string)))
        elif isinstance(value, int):
            graph.add((individual_uri, ENDE_NAMESPACE[prop], Literal(value, datatype=XSD.integer)))
        elif isinstance(value, float):
            graph.add((individual_uri, ENDE_NAMESPACE[prop], Literal(value, datatype=XSD.float)))

# Função para processar um arquivo CSV
def process_csv_file(file_path):
    microdados = pd.read_csv(file_path, sep=';', encoding='utf-8')
    for _, row in microdados.iterrows():
        aluno_id = f"aluno_{row['ID_ALUNO']}_{row['NU_ANO']}"
        add_instance(g, 'Aluno', aluno_id, {
            'idAluno': aluno_id,
            'ano': row['NU_ANO']
        })

        curso_id = f"curso_{row['CO_CURSO']}_{row['NU_ANO']}"
        add_instance(g, 'Curso', curso_id, {
            'codigoCurso': row['CO_CURSO']
        })

        prova_id = f"prova_{row['NU_ANO']}"
        add_instance(g, 'Prova', prova_id, {
            'ano': row['NU_ANO']
        })

        resposta_id = f"resposta_{row['ID_ALUNO']}_{row['NU_ANO']}"
        add_instance(g, 'Resposta', resposta_id, {
            'acertosOFG': row['DS_VT_ACE_OFG'],
            'acertosOCE': row['DS_VT_ACE_OCE']
        })

# Processar todos os arquivos CSV
for year, path in csv_paths.items():
    process_csv_file(path)

# Salvar a ontologia atualizada
updated_ontology_path = '/content/drive/MyDrive/Mestrado/TrabalhoESeBD/ontologia/enade_atualizada.owl'
g.serialize(destination=updated_ontology_path, format='xml')

print(f"Ontologia atualizada salva em: {updated_ontology_path}")


Erro ao carregar ontologia: file:///content/drive/MyDrive/Mestrado/TrabalhoESeBD/ontologia/enade.owl:68:6: Invalid node element URI: http://www.w3.org/1999/02/22-rdf-syntax-ns#li
Ontologia atualizada salva em: /content/drive/MyDrive/Mestrado/TrabalhoESeBD/ontologia/enade_atualizada.owl


In [3]:
!pip install rdflib
# Importar Literal da rdflib
from rdflib import Graph, Namespace, RDF, Literal
import pandas as pd

# Caminho para a ontologia fornecida
ontology_path = '/content/drive/MyDrive/Mestrado/TrabalhoESeBD/ontologia/enade_atualizada.owl'

# Carregar a ontologia
g = Graph()
g.parse(ontology_path, format='xml')

# Definir o namespace da ontologia
ENDE_NAMESPACE = Namespace('http://example.org/ontologies/enade#')

# Listas para armazenar os dados extraídos
alunos_data = []
cursos_data = []
provas_data = []
respostas_data = []

# Extrair instâncias de Aluno
for aluno in g.subjects(RDF.type, ENDE_NAMESPACE.Aluno):
    aluno_id = g.value(aluno, ENDE_NAMESPACE.idAluno)
    ano = g.value(aluno, ENDE_NAMESPACE.ano)
    # Tentar converter o ano para inteiro, tratando-o como float primeiro, se necessário
    try:
        ano = int(float(ano))  # Converte primeiro para float, depois para int
    except (ValueError, TypeError):
        ano = None  # Define como None se a conversão falhar
    alunos_data.append({'idAluno': str(aluno_id), 'ano': ano})

# Extrair instâncias de Curso
for curso in g.subjects(RDF.type, ENDE_NAMESPACE.Curso):
    codigo_curso = g.value(curso, ENDE_NAMESPACE.codigoCurso)
    cursos_data.append({'codigoCurso': str(codigo_curso)})

# Extrair instâncias de Prova
for prova in g.subjects(RDF.type, ENDE_NAMESPACE.Prova):
    ano_prova = g.value(prova, ENDE_NAMESPACE.ano)
    try:
        ano_prova = int(float(ano_prova))
    except (ValueError, TypeError):
        ano_prova = None
    provas_data.append({'ano': ano_prova})

# Extrair instâncias de Resposta
for resposta in g.subjects(RDF.type, ENDE_NAMESPACE.Resposta):
    acertos_ofg = g.value(resposta, ENDE_NAMESPACE.acertosOFG)
    acertos_oce = g.value(resposta, ENDE_NAMESPACE.acertosOCE)

    # Debug: Mostrar os valores extraídos
    print(f"Resposta: {resposta}")
    print(f"  Acertos OFG (bruto): {acertos_ofg}")
    print(f"  Acertos OCE (bruto): {acertos_oce}")

    # Verificar e converter valores
    try:
        acertos_ofg = float(acertos_ofg) if acertos_ofg is not None else None
        acertos_oce = float(acertos_oce) if acertos_oce is not None else None
    except (ValueError, TypeError):
        acertos_ofg = acertos_oce = None

    # Adicionar os valores ao dicionário
    respostas_data.append({
        'acertosOFG': acertos_ofg,
        'acertosOCE': acertos_oce
    })

# Converter os dados extraídos em DataFrames para análise
alunos_df = pd.DataFrame(alunos_data)
cursos_df = pd.DataFrame(cursos_data)
provas_df = pd.DataFrame(provas_data)
respostas_df = pd.DataFrame(respostas_data)

# Mostrar as primeiras linhas dos DataFrames extraídos
alunos_df.head(), cursos_df.head(), provas_df.head(), respostas_df.head()


A saída de streaming foi truncada nas últimas 5000 linhas.
  Acertos OFG (bruto): 5.0
  Acertos OCE (bruto): 11.0
Resposta: http://example.org/ontologies/enade#resposta_1309015.0_2018.0
  Acertos OFG (bruto): 4.0
  Acertos OCE (bruto): 3.0
Resposta: http://example.org/ontologies/enade#resposta_1309015.0_2022.0
  Acertos OFG (bruto): 0.0
  Acertos OCE (bruto): 0.0
Resposta: http://example.org/ontologies/enade#resposta_1309016.0_2015.0
  Acertos OFG (bruto): 4.0
  Acertos OCE (bruto): 10.0
Resposta: http://example.org/ontologies/enade#resposta_1309016.0_2018.0
  Acertos OFG (bruto): 3.0
  Acertos OCE (bruto): 6.0
Resposta: http://example.org/ontologies/enade#resposta_1309016.0_2022.0
  Acertos OFG (bruto): 0.0
  Acertos OCE (bruto): 0.0
Resposta: http://example.org/ontologies/enade#resposta_1309017.0_2015.0
  Acertos OFG (bruto): 6.0
  Acertos OCE (bruto): 9.0
Resposta: http://example.org/ontologies/enade#resposta_1309017.0_2018.0
  Acertos OFG (bruto): 2.0
  Acertos OCE (bruto): 7.0
Res

(                    idAluno   ano
 0   aluno_11053961.0_2022.0  2022
 1  aluno_110539610.0_2022.0  2022
 2  aluno_110539611.0_2022.0  2022
 3  aluno_110539612.0_2022.0  2022
 4  aluno_110539613.0_2022.0  2022,
   codigoCurso
 0   1105396.0
 1   1132074.0
 2   1132074.0
 3   1132074.0
 4     13089.0,
     ano
 0  2015
 1  2018
 2  2022,
    acertosOFG  acertosOCE
 0         0.0         0.0
 1         0.0         0.0
 2         0.0         0.0
 3         0.0         0.0
 4         0.0         0.0)

In [4]:
!pip install rdflib
from rdflib import Graph, Namespace, RDF
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import re
import matplotlib.pyplot as plt

# Caminho para a ontologia fornecida
ontology_path = '/content/drive/MyDrive/Mestrado/TrabalhoESeBD/ontologia/enade_atualizada.owl'

# Carregar a ontologia
g = Graph()
g.parse(ontology_path, format='xml')

# Definir o namespace da ontologia
ENDE_NAMESPACE = Namespace('http://example.org/ontologies/enade#')

# Listas para armazenar os dados extraídos
alunos_data = []
cursos_data = []
provas_data = []
respostas_data = []

# Função para extrair informações da URL
def extrair_info_url(url):
    # Corrigir a expressão regular para capturar os grupos corretamente
    match = re.match(r'http://example.org/ontologies/enade#resposta_(\d+)\.(\d+)_(\d+)\.(\d+)', url)
    if match:
        codigo_curso = match.group(1)
        id_aluno = match.group(3)
        ano = float(match.group(4))  # Convertido para float diretamente
        return codigo_curso, id_aluno, ano
    return None, None, None

# Extrair instâncias de Resposta
for resposta in g.subjects(RDF.type, ENDE_NAMESPACE.Resposta):
    url_resposta = str(resposta)  # Garantir que a URL seja uma string
    codigo_curso, id_aluno, ano = extrair_info_url(url_resposta)

    acertos_ofg = g.value(resposta, ENDE_NAMESPACE.acertosOFG)
    acertos_oce = g.value(resposta, ENDE_NAMESPACE.acertosOCE)

    # Verificar e converter valores
    try:
        acertos_ofg = float(acertos_ofg) if acertos_ofg is not None else None
        acertos_oce = float(acertos_oce) if acertos_oce is not None else None
    except (ValueError, TypeError):
        acertos_ofg = acertos_oce = None

    # Adicionar os valores ao dicionário
    respostas_data.append({
        'codigoCurso': codigo_curso,
        'idAluno': id_aluno,
        'ano': ano,
        'acertosOFG': acertos_ofg,
        'acertosOCE': acertos_oce
    })

# Converter os dados extraídos em DataFrames para análise
alunos_df = pd.DataFrame(alunos_data)
cursos_df = pd.DataFrame(cursos_data)
provas_df = pd.DataFrame(provas_data)
respostas_df = pd.DataFrame(respostas_data)

# Mostrar as primeiras linhas dos DataFrames extraídos
print("Dados extraídos:")
print(alunos_df.head())
print(cursos_df.head())
print(provas_df.head())
print(respostas_df.head())

# Identificar cursos únicos
cursos = respostas_df['codigoCurso'].unique()

# Dicionário para armazenar previsões
previsoes_2026 = {}

# Processar cada curso individualmente
for codigo_curso in cursos:
    # Verificar se o código do curso é válido
    if pd.isna(codigo_curso):
        #print(f"Código do curso é None. Verifique os dados extraídos.")
        continue

    # Filtrar dados para o curso atual
    dados_curso = respostas_df[respostas_df['codigoCurso'] == codigo_curso]

    # Verificar se há dados suficientes para treinamento
    if len(dados_curso) < 2:
        #print(f"Não há dados suficientes para o curso com código {codigo_curso}.")
        continue

    # Remover ou preencher valores NaN
    dados_curso = dados_curso.dropna()  # ou use fillna() conforme necessário

    # Verificar se há dados após limpeza
    if dados_curso.empty:
        print(f"Após limpeza, não há dados suficientes para o curso com código {codigo_curso}.")
        continue

    # Definir características (X) e alvo (y)
    X = dados_curso[['ano']]
    y_ofg = dados_curso['acertosOFG']
    y_oce = dados_curso['acertosOCE']

    # Criar e treinar o modelo de regressão linear para OFG
    model_ofg = LinearRegression()
    model_ofg.fit(X, y_ofg)

    # Criar e treinar o modelo de regressão linear para OCE
    model_oce = LinearRegression()
    model_oce.fit(X, y_oce)

    # Fazer previsões para 2026
    ano_2026 = pd.DataFrame({'ano': [2026]})
    acertos_2026_ofg = model_ofg.predict(ano_2026)
    acertos_2026_oce = model_oce.predict(ano_2026)

    # Avaliar o modelo para OFG
    y_pred_ofg = model_ofg.predict(X)
    mse_ofg = mean_squared_error(y_ofg, y_pred_ofg)

    # Avaliar o modelo para OCE
    y_pred_oce = model_oce.predict(X)
    mse_oce = mean_squared_error(y_oce, y_pred_oce)

    # Armazenar previsão e erro para o curso
    previsoes_2026[codigo_curso] = {
        'acertosOFG_previsto': acertos_2026_ofg[0],
        'acertosOCE_previsto': acertos_2026_oce[0],
        'mse_ofg': mse_ofg,
        'mse_oce': mse_oce
    }

# Exibir previsões
for codigo_curso, info in previsoes_2026.items():
    print(f"Código do Curso: {codigo_curso}")
    print(f"  Acertos OFG previstos para 2026: {info['acertosOFG_previsto']:.2f}")
    print(f"  Acertos OCE previstos para 2026: {info['acertosOCE_previsto']:.2f}")
    print(f"  Mean Squared Error OFG: {info['mse_ofg']:.2f}")
    print(f"  Mean Squared Error OCE: {info['mse_oce']:.2f}")


Dados extraídos:
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
  codigoCurso idAluno  ano  acertosOFG  acertosOCE
0    11053961    2022  0.0         0.0         0.0
1   110539610    2022  0.0         0.0         0.0
2   110539611    2022  0.0         0.0         0.0
3   110539612    2022  0.0         0.0         0.0
4   110539613    2022  0.0         0.0         0.0
Código do Curso: 11320741
  Acertos OFG previstos para 2026: 1.33
  Acertos OCE previstos para 2026: 3.00
  Mean Squared Error OFG: 1.56
  Mean Squared Error OCE: 8.67
Código do Curso: 113207410
  Acertos OFG previstos para 2026: 2.67
  Acertos OCE previstos para 2026: 4.67
  Mean Squared Error OFG: 3.56
  Mean Squared Error OCE: 16.89
Código do Curso: 113207411
  Acertos OFG previstos para 2026: 2.67
  Acertos OCE previstos para 2026: 5.33
  Mean Squared Error OFG: 3.56
  Mean Squared Error OCE: 24.89
Código do Curso: 113207412
  Acertos OFG previstos para